# Exploring tensor decompositions in pytorch
Simon Aertssen
17/03/2020

In [1]:
import torch
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


This notebook is an exploration of the different functions and methods necessary to perform the CANDECOMP/PARAFAC and Tensor-Train Decompositions. These will be gathered in a ```polution``` package later. The ```tensorly```package is a great resource: https://github.com/tensorly/tensorly/tree/master/tensorly

Fold/ Unfold 
https://github.com/tensorly/tensorly/blob/c1a3051f43b11350ee29c28cc81ec2cd2c60e99c/tensorly/base.py#L54

MoveAxis method
https://github.com/tensorly/tensorly/blob/0a89edd824431f66d5a76c1a3e097a80c8475978/tensorly/backend/pytorch_backend.py#L90

Parafac:
https://github.com/tensorly/tensorly/blob/master/tensorly/decomposition/candecomp_parafac.py

## 1. Useful products:

The $\textbf{Matrix Kronecker product}$ is defined as

\begin{align*} \boldsymbol{P}^{\:\mathrm{I} \times \mathrm{J}} \otimes \boldsymbol{Q}^{\:\mathrm{K} \times \mathrm{L}}=\boldsymbol{R}^{\:\mathrm{IK} \times \mathrm{JL}}, \text { such that } r_{k+K(i-1), \: l+L(j-1)}=p_{i j} q_{k l} \end{align*}

Naive: use loops. Smart: reshape P and Q so that their dimensions do not overlap, then the elementwise or Hadamard product gives the desired result.

See [Wikipedia](https://en.wikipedia.org/wiki/Kronecker_product) for the examples.


In [2]:
def KronProd2D(P, Q):
    # Register dimensions.
    I, J = P.shape
    K, L = Q.shape
    
    # Adjust dimensions of P and Q to perform smart multiplication:
    # interweave the dimensions containing values and perform elementwise multiplication.
    P = P.view(I, 1, J, 1)
    Q = Q.view(1, K, 1, L)
    
    R = P * Q
    return R.view(I*K, J*L)

def KronProd(P, Q):
    # This should work for higher order tensors.
    # Register and check dimensions.
    pshape = P.shape
    qshape = Q.shape
    if P.dim() != Q.dim():
        raise ValueError('Matrices should be of the same order: P.dim() =' + str(P.dim()) + ' != Q.dim() = ' + str(Q.dim()))
    
    # Adjust dimensions of P and Q to perform smart multiplication:
    # interweave the dimensions containing values and perform elementwise multiplication.
    # Start with a list of ones and set dimensions as every even or uneven index.
    pindices = [1]*2*len(pshape)    
    pindices[::2] = pshape
    qindices = [1]*2*len(qshape)
    qindices[1::2] = qshape
        
    P = P.view(pindices)
    Q = Q.view(qindices)
    
    R = P * Q
    rshape = [p*q for p, q in zip(pshape,qshape)]
    return R.view(rshape)

Example:

In [3]:
P = torch.tensor([[1, -4, 7], [-2, 3, 3]])
Q = torch.tensor([[8, -9, -6, 5], [1, -3, -4, 7], [2, 8, -8, -3], [1, 2, -5, -1]])
R = KronProd(P, Q)
assert torch.all(R.eq(KronProd2D(P, Q)))

The $\textbf{Kahtri-Rhao product}$ is defined as

\begin{align*}
\boldsymbol{P}^{\:\mathrm{I} \times \mathrm{J}}|\otimes| \boldsymbol{Q}^{\mathrm{K} \times \mathrm{J}}=\boldsymbol{P}^{\:\mathrm{I} \times \mathrm{J}} \odot \boldsymbol{Q}^{\mathrm{K} \times \mathrm{J}}=\boldsymbol{R}^{\:\mathrm{IK} \times \mathrm{J}}, \text { such that } r_{k+K(i-1), j}=p_{i j} q_{k j}
\end{align*}

This can be seen as a column-wise kronecker product.

In [4]:
def KathRaoProd(P, Q):
    # Register and check dimensions
    pshape = P.shape
    qshape = Q.shape
    J = pshape[-1]
    if J != qshape[-1]:
        raise ValueError('Matrices should have the same number of columns: ' + str(J) + ' != ' + str(qshape[-1]))
    
    # Make R an empty tensor
    rshape = [p*q for p, q in zip(pshape,qshape)]
    rshape[-1] = J
    R = torch.zeros(rshape)
    for j in range(J):
        R[:,j] = KronProd(P[:,j], Q[:,j])
                
    # Tried with slicing but did not seem to work:
    # column_indices = list(range(J)) or slice(0,J)
    # R[:, column_indices] = KronProd(P[:, column_indices], Q[:, column_indices])
    return R


Example:

In [5]:
P = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
Q = torch.tensor([[1, 4, 7], [2, 5, 8], [3, 6, 9]])
R = KathRaoProd(P, Q)
#print(R)

## 2. Mode-m Matricization

For some of the decompositions we will need tensor unfolding. The mode-$n$ matricization of a tensor $\mathcal{X} \in \mathrm{R}^{I_{1} \times I_{2} \times \ldots \times I_{N}}$ is denoted $\boldsymbol{X}_{(n)}$ and arranges the mode-$n$ fibres to be the columns of the resulting matrix.

\begin{align*}
\mathcal{X}^{I_{1} \times I_{2} \times \ldots \times I_{N}} \hspace{5mm}
\overset{\underset{\text{unfolding}}{\longrightarrow}}{\underset{\text{folding}}{\longleftarrow}} \hspace{5mm}
\boldsymbol{X}_{(n)}^{I_{n} \times I_{1} \cdot I_{2} \cdots I_{n-1} \cdot I_{n+1} \cdots I_{N}} 
\end{align*}

While writing the function underneath (```Unfold```) it became clear that Kolda's definition was lacking some of the intuition behind torch tensors and their indexing. A good discussion can be found here: https://jeankossaifi.com/blog/unfolding.html

In [6]:
def Permute_tensor(tensor, origin, axis, outputshape=None):
    # Perform the permutation of the tensor here, to be used in both Fold and Unfold.
    # For more explanation on this function see Unfold().
    
    dims = list(range(tensor.dim()))
    # For later use, we want to be able to map indices for a circular permutation
    if origin < 0: origin = dims[origin]
    if axis < 0: axis = dims[axis]
            
    dims.pop(origin)
    dims = dims[::-1]
    dims.insert(axis, origin)
    tensor = tensor.permute(*dims)
    
    if outputshape:
        tensor = tensor.reshape(outputshape)
    return tensor


def Unfold(tensor, mode):
    # Notation sets minimal mode to be 1, but we want to map that to zero. We also cannot mode_n unfold a matrix with n-1 modes.
    if mode > len(tensor.shape) or mode < 0:
        raise ValueError('Tensor has order ' + str(len(tensor.shape)) + ', cannot give mode ' + str(mode))
    
    # Register tensor shape and mode size
    order = tensor.dim()
    tshape = tensor.shape    
    I_n = tshape[mode]
    
    # To get the prescribed matrix in Kolda et al, we need to perform a permutation of the axes.
    # The selected mode needs to be viewed as the first, and all other dimensions should reverse their order.
    # It took a lot of experiments to get this right, the full functionality has been moved to Permute_tensor().    
    return Permute_tensor(tensor, mode, 0, (I_n, -1))


In [23]:
# Following the example on p460:
# This definition of X gives wrong dimensions:
#X = torch.tensor([[ [1, 4, 7, 10], [2, 5, 8, 11], [3, 6, 9, 12] ], [ [13, 16, 19, 22], [14, 17, 20, 23], [15, 18, 21, 24] ]])

X = torch.tensor([[ [1,13], [4,16], [7,19], [10,22] ], [ [2,14], [5,17], [8,20], [11,23] ], [ [3,15], [6,18], [9,21], [12,24] ]])
print("X.shape = " + str(X.shape))
X1 = X[...,0]
X2 = X[...,1]
print("X_1 = \n" + str(X1.numpy()))
print("X_2 = \n" + str(X2.numpy()))
print("X = \n" + str(X.numpy()))


X.shape = torch.Size([3, 4, 2])
X_1 = 
[[ 1  4  7 10]
 [ 2  5  8 11]
 [ 3  6  9 12]]
X_2 = 
[[13 16 19 22]
 [14 17 20 23]
 [15 18 21 24]]
X = 
[[[ 1 13]
  [ 4 16]
  [ 7 19]
  [10 22]]

 [[ 2 14]
  [ 5 17]
  [ 8 20]
  [11 23]]

 [[ 3 15]
  [ 6 18]
  [ 9 21]
  [12 24]]]


In [28]:
print(Unfold(X, 0).numpy())
print(Unfold(X, 1).numpy())
print(Unfold(X, 2).numpy())
# Note that the .numpy() is only here to represent the unfolded tensors in a nice format.

[[ 1  4  7 10 13 16 19 22]
 [ 2  5  8 11 14 17 20 23]
 [ 3  6  9 12 15 18 21 24]]
[[ 1  2  3 13 14 15]
 [ 4  5  6 16 17 18]
 [ 7  8  9 19 20 21]
 [10 11 12 22 23 24]]
[[ 1  2  3  4  5  6  7  8  9 10 11 12]
 [13 14 15 16 17 18 19 20 21 22 23 24]]


Do we need a ```Fold``` function? Update: yes. Perform reverse operations of ```Unfold```.

In [9]:
def Fold(unfolded_tensor, mode, desired_shape):
    # Perform the same entry safety controls
    if mode > len(desired_shape) or mode < 0:
        raise ValueError('Folded tensor has order ' + str(len(desired_shape)) + ', cannot give mode ' + str(mode))
    
    # Before calling Permute_tensor(), we need to reshape the unfolded tensor back to its' desired order.
    # This does not yield the desired tensor but takes into account the coming permutation.
    shape = list(desired_shape)
    axis = shape.pop(mode)
    shape = shape[::-1]
    shape.insert(0, axis)
    tensor = unfolded_tensor.reshape(shape)
    
    # return Permute_tensor(tensor, 0, mode, desired_shape)
    # Exactly my point before, we have accounted for the exact inverse transformation without the need for a reshape.
    # Both return statements yield the same result.
    return Permute_tensor(tensor, 0, mode)


In [10]:
for mode in range(3):
    FoldedX = Fold(Unfold(X, mode), mode, X.shape)
    print("FoldedX_", mode, "=\n", FoldedX.numpy())
    assert torch.all(X.eq(FoldedX))
# If no errors, all are exactly equal to X


FoldedX_ 0 =
 [[[ 1 13]
  [ 4 16]
  [ 7 19]
  [10 22]]

 [[ 2 14]
  [ 5 17]
  [ 8 20]
  [11 23]]

 [[ 3 15]
  [ 6 18]
  [ 9 21]
  [12 24]]]
FoldedX_ 1 =
 [[[ 1 13]
  [ 4 16]
  [ 7 19]
  [10 22]]

 [[ 2 14]
  [ 5 17]
  [ 8 20]
  [11 23]]

 [[ 3 15]
  [ 6 18]
  [ 9 21]
  [12 24]]]
FoldedX_ 2 =
 [[[ 1 13]
  [ 4 16]
  [ 7 19]
  [10 22]]

 [[ 2 14]
  [ 5 17]
  [ 8 20]
  [11 23]]

 [[ 3 15]
  [ 6 18]
  [ 9 21]
  [12 24]]]


## 3. CP decomposition


Use the alternating least squares approach. Start with random matrices.

In [11]:
def CPD(tensor, rank=5, maxiter=100):
    # Problems were recorded with tensors that had only type = torch.LongTensor.
    # Reformat into torch.FloatTensor, and clone to avoid performing CPD twice on the same tensor.
    tensor = tensor.type(torch.FloatTensor).clone()
    
    # Note that we will perform n_iter_max iterations per tensor order!
    # Register input variables:
    tshape = tensor.shape
    order = tensor.dim()
    
    # Initialization: create a list of matrices to represent the input tensor: tensor = [t_1, t_2, .. , t_order]
    # Incorrect: ABC = [torch.rand((rank, rank))]*order
    ABC = []
    for o in range(order):
        ABC.append(torch.rand((tshape[o], rank)))
    
    # Main loop: for every iteration, adjust every factor matrix
    for iteration in range(maxiter):
        for i, fmatrix in enumerate(ABC):
            # Get all the other fmatrices, temporarily remove the element:
            ABC.pop(i)

            # Construct V: use the first element ABC[0] and the loop over everything after with ABC[1::]
            V = ABC[0].t() @ ABC[0]
            # pinverse_V = torch.ones((rank, rank), dtype=tensor.dtype)
            for ABC_notfmatrix in ABC[1::]:
                V = V * (ABC_notfmatrix.t() @ ABC_notfmatrix)
                # pinverse_V = pinverse_V * (fmatrix.t() @ fmatrix)
            
            # Construct W (kr product of all matrices other way around):
            # take the last element of ABC and then loop over ABC[::-1] excluding the last element.
            # Example: ABC = [1,2,3,4] and ABC[::-1][1::] = [3,2,1] and ABC[1::-1] = [2,1]
            # This would be easier with a pop and insert, but we need to recycle ABC a lot...
            W = ABC[-1]
            for ABC_notfmatrix in ABC[::-1][1::]:
                W = KathRaoProd(W, ABC_notfmatrix)
                
            # Here is a problem:
            fmatrix = Unfold(tensor, i) @ W @ torch.pinverse(V)
            # Other tries to get the p_inverse
            # pinverse_V = ((V.t() @ V).inverse()) @ V.t()
            # pinverse_V = V.t() @ ((V.t() @ V).inverse()) 
            # fmatrix = Unfold(tensor, i) @ W @ pinverse_V
            
            # Push the lost fmatrix back in:
            ABC.insert(i, fmatrix)
    return ABC

In [19]:
# test:
tensor = torch.rand((10,15,20))
print(tensor.type())
factor_matrices = CPD(tensor)
print(len(factor_matrices), "factor matrices")

for fm in factor_matrices:
    print(fm.shape)

torch.FloatTensor
3 factor matrices
torch.Size([10, 5])
torch.Size([15, 5])
torch.Size([20, 5])


In [13]:
def UnfoldFM(factor_matrices, mode):
    if mode > len(factor_matrices) or mode < 0:
        raise ValueError('Tensor has order ' + str(len(factor_matrices)) + ', cannot give mode ' + str(mode))
    # Start with the first element (for normal dot product), then the last element is ready for the backward loop.
    first = factor_matrices.pop(mode)
    prod = factor_matrices.pop(-1)
    for factor in factor_matrices[::-1]:
        prod = KathRaoProd(prod, factor)
    return first @ prod.t()

In [14]:
factor_matrices = CPD(X, 2)
print(UnfoldFM(factor_matrices, 0))
print(Unfold(X, 0))

tensor([[ 0.9602,  3.9909,  7.0215, 10.0522, 13.2199, 16.0837, 18.9474, 21.8112],
        [ 2.0001,  5.0003,  8.0005, 11.0006, 13.9877, 16.9947, 20.0018, 23.0088],
        [ 3.0399,  6.0097,  8.9794, 11.9491, 14.7554, 17.9058, 21.0561, 24.2065]])
tensor([[ 1,  4,  7, 10, 13, 16, 19, 22],
        [ 2,  5,  8, 11, 14, 17, 20, 23],
        [ 3,  6,  9, 12, 15, 18, 21, 24]])


In [15]:
def Compose(factors):
    # Make copy to remove dependancy
    factor_matrices = []
    for f in factors:
        factor_matrices.append(f.clone())
    
    # Register dimensions of the tensor:
    order = len(factor_matrices)
    tshape = [x.shape[0] for x in factor_matrices]
    
    # Use the mode-0 unfolding and just fold it back.
    return Fold(UnfoldFM(factor_matrices, 0), 0, tshape)

In [16]:
tensor = torch.rand((10,15,20))
factor_matrices = CPD(tensor, 100)
test = Compose(factor_matrices)

print(torch.norm(tensor))
print(torch.norm(test))
print(torch.norm(tensor - test))

tensor(31.7990)
tensor(31.7990)
tensor(0.0005)


In [17]:
lmtns = tensor.shape
print("The tensor has", lmtns[0]*lmtns[1]*lmtns[2], "elements.")

fmatrixdims = 1
for f in factor_matrices:
    fmatrixdims *= f.shape[0]*f.shape[1]
    
print("The factor matrices have", fmatrixdims, "elements.")


The tensor has 3000 elements.
The factor matrices have 3000000000 elements.
